In [1]:
!python --version

Python 3.10.11


# Lightweight Fine-Tuning

<font color="red"> 

- PEFT technique: LoRA (Low-Rank Adaptation)
- Model: `distilbert-base-uncased`
- Evaluation approach: Accuracy and F1-score
- Fine-tuning dataset: IMDb Reviews (binary sentiment classification)

</font>

In [2]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

import torch
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments

from peft import get_peft_model, LoraConfig, AutoPeftModelForSequenceClassification

## Helpful Functions



In [3]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(preds, labels)
    f1 = f1_score(preds, labels, average="weighted")
    return {"accuracy": acc, "f1": f1}

## Loading and Evaluating a Foundation Model

In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [4]:
# Load pretrained model and tokenizer
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name
)

/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Load dataset
dataset = load_dataset("imdb", )
dataset = dataset.map(preprocess_function, batched=True)

# Ensure that the model receives inputs in the correct format for training and evaluation
dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [6]:
# Evaluate the pretrained model
trainer = Trainer(
    model=model,
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)

eval_results = trainer.evaluate()
print(f"Evaluation for pretrained model: {eval_results}")

Evaluation for pretrained model: {'eval_loss': 0.6977689266204834, 'eval_accuracy': 0.4996, 'eval_f1': 0.66597346981213, 'eval_runtime': 412.746, 'eval_samples_per_second': 60.57, 'eval_steps_per_second': 7.571}


## Performing Parameter-Efficient Fine-Tuning

In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [7]:
# Create PEFT model
peft_config = LoraConfig(
    task_type="SEQ_CLS",
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"]
)

peft_model = get_peft_model(model, peft_config)

In [8]:
# Check trainable parameters of the PEFT model
peft_model.print_trainable_parameters()

trainable params: 1,331,716 || all params: 67,694,596 || trainable%: 1.967241225577297


In [9]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs"
)

# Train the PEFT model
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.257000,0.237407,0.903920,0.903998
2,0.243400,0.218991,0.912760,0.912815
3,0.225800,0.209126,0.918240,0.918241
4,0.212900,0.217593,0.918920,0.918933
5,0.200100,0.202390,0.923040,0.923040
6,0.198300,0.213539,0.923320,0.923323
7,0.186900,0.215877,0.922760,0.922769
8,0.185900,0.208203,0.923920,0.923920
9,0.182300,0.214719,0.924080,0.924084
10,0.179000,0.216512,0.923840,0.923840


TrainOutput(global_step=15630, training_loss=0.2109471498661444, metrics={'train_runtime': 13721.6774, 'train_samples_per_second': 18.219, 'train_steps_per_second': 1.139, 'total_flos': 3.3684851712e+16, 'train_loss': 0.2109471498661444, 'epoch': 10.0})

In [10]:
# Save fine-tuned model
peft_model.save_pretrained("./peft_model")
print("Fine-tuned model saved!")

Fine-tuned model saved!


## Performing Inference with a PEFT Model

In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [11]:
# Load fine-tuned PEFT model
peft_model = AutoPeftModelForSequenceClassification.from_pretrained("./peft_model/")
print("Fine-tuned model loaded!")

/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fine-tuned model loaded!


In [12]:
# Evaluate the fine-tuned model
trainer = Trainer(
    model=peft_model,
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)

eval_results_peft = trainer.evaluate()
print(f"Evaluation for fine-tuned model: {eval_results_peft}")

Evaluation for fine-tuned model: {'eval_loss': 0.21651224792003632, 'eval_accuracy': 0.92384, 'eval_f1': 0.9238403821423334, 'eval_runtime': 425.7198, 'eval_samples_per_second': 58.724, 'eval_steps_per_second': 7.341}


<font color="red"> 

Fine-tuning with PEFT significantly improved model performance, reducing the evaluation loss from **0.698** to **0.217** while boosting accuracy from **49.96%** to **92.38%** and F1-score from **66.60%** to **92.38%**. These results indicate that fine-tuning effectively adapted the model to the IMDb sentiment classification task, leading to much better predictive capability while maintaining a similar evaluation runtime.

</font>